In [1]:
import os
import sys
from datasets import load_dataset
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


/Users/roshanjeyakumar/elve/myBPE/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("stanfordnlp/imdb")
train_data = ds['train']
test_data = ds['test']
X_train = train_data['text']
y_train = train_data['label']
X_test = test_data['text']
y_test = test_data['label']


In [3]:
Tokenizer = Tokenizer(num_words=100000)
Tokenizer.fit_on_texts(X_train)
maxlen = max([len(x.split()) for x in X_train])
X_train = Tokenizer.texts_to_sequences(X_train)
X_test = Tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)


In [4]:
X_train_tensor = tf.convert_to_tensor(X_train)
y_train_tensor = tf.convert_to_tensor(y_train)
X_test_tensor = tf.convert_to_tensor(X_test)
y_test_tensor = tf.convert_to_tensor(y_test)

In [27]:
def embedding_model():
    vocab_size = min(Tokenizer.num_words,len(Tokenizer.word_index) + 1)
    embedding_dim = 1024
    model = keras.Sequential([
        keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=2470),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.LSTM(32, return_sequences=True),
        keras.layers.Dropout(0.2),
        keras.layers.LSTM(16),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    return model


embedding_model = embedding_model()
embedding_model.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

embedding_model.fit(X_train_tensor, y_train_tensor, epochs=4, batch_size=1024, validation_data=(X_test_tensor, y_test_tensor), verbose=1)
embedding_model.save('imdb_embedding_model.h5')


Epoch 1/4


ValueError: Input 0 of layer "lstm_27" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 1024)

In [24]:
print('accuracy:', embedding_model.evaluate(X_test_tensor, y_test_tensor, verbose=0)[1])


accuracy: 0.8657199740409851
